## Define the Objective of the Analysis

...

Analisi del sentiment relativo al dibattito sulla demolizione dello stadio di San Siro a Milano e individuazione di eventuali 'influencer' sul tema.

## Collect Data

In [1]:
# imports
import numpy as np
import pandas as pd
import tweepy

import re
import string
import itertools
from collections import Counter
from datetime import datetime
from numpy.core.multiarray import result_type
import time
tweepy.__version__

'4.8.0'

In [2]:
# insert the keys here
consumer_key = 'VPzjkqKl2y1uSTJQvnVqS9e1X' 
consumer_secret = 'STG2IzVMf65vPGeOvBQyzdeoKBExAr5sIkhOaBeDe2fnIN14vY'
access_token = '1508409949835214853-HIyZJ3oT32TijKsdNDhGFZEEQTWwau'
access_token_secret = 'uLcs9hUYmLdocxkaSfXo69Gii46TISu5qZj5F6f6fBfnW'

#### Tweets Download

The next step is creating an OAuthHandler instance. We pass our consumer key and access token which we defined above.

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Next, we pass the OAuthHandler instance into the API method.

In [4]:
api = tweepy.API(auth)

Tweets that contain a specific hashtag

In [ ]:
'''
hashtag = "#SanSiro"

list_tweets = []

for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, count=100, lang='it').items(500):
  
  full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']

  list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
  tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])                           

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.
'''

In [ ]:
'''
# Turn list_tweet into a DataFrame changing column names
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted', 'user_mentions', 'hastags'])
tweets.to_csv('../data/SanSiro.csv')
'''

In [6]:
#tweets = pd.read_csv('../data/SanSiro.csv')
#tweets = tweets.drop('Unnamed: 0', axis=1) 
tweets.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags
0,2022-12-24 09:51:02+00:00,1606588252269318144,RT @paolopileri_: In Lombardia fioccano candid...,0,5,comitatogoccia,Milano quartiere Bovisa,False,"[{'screen_name': 'paolopileri_', 'name': 'Paol...",[]
1,2022-12-24 08:00:49+00:00,1606560513927913473,RT @paolopileri_: In Lombardia fioccano candid...,0,5,pa_poz,NaN,False,"[{'screen_name': 'paolopileri_', 'name': 'Paol...",[]
2,2022-12-24 07:48:15+00:00,1606557349434609666,RT @paolopileri_: In Lombardia fioccano candid...,0,5,iggyHT,milano e trallallero,False,"[{'screen_name': 'paolopileri_', 'name': 'Paol...",[]
3,2022-12-23 21:05:36+00:00,1606395621958733824,RT @metaanto: Bravo @VittorioSgarbi il PD vuol...,0,5,CedroneBruno,Il paese delle meraviglie,False,"[{'screen_name': 'metaanto', 'name': 'antoniet...","[{'text': 'SanSiro', 'indices': [131, 139]}]"
4,2022-12-23 20:50:04+00:00,1606391715438465038,RT @TamminenJuha: Tifosi del FC Internazionale...,0,4,StatusSymbol2,"Milano, Italia",False,"[{'screen_name': 'TamminenJuha', 'name': 'Juha...","[{'text': 'tifosi', 'indices': [82, 89]}, {'te..."


In [7]:
tweets.shape

(209, 10)

In [8]:
tweets.columns

Index(['date', 'id', 'text', 'like', 'n_rt', 'author', 'location', 'retweeted',
       'user_mentions', 'hastags'],
      dtype='object')

#### Data Pre-Processing

In [9]:
# Remove duplicates
tweets.drop_duplicates(subset ="id", inplace = True)
tweets.reset_index(drop = True, inplace = True)
tweets.shape

(209, 10)

In [10]:
# Change date format
day = tweets['date'].dt.day
month = tweets['date'].dt.month
year = tweets['date'].dt.year

date = year.astype(str) + month.astype(str).str.zfill(2) + day.astype(str).str.zfill(2)
date = pd.to_datetime(date, format='%Y%m%d')
tweets.drop(columns = ['date'], inplace = True)
tweets['date'] = date

# Reorder columns
cols = tweets.columns.tolist()
cols = cols[-1:] + cols[:-1]
tweets = tweets[cols].copy()

print('Tweet per day:')
print()
print(tweets.groupby('date').count()['id'])
# print()
# print()
# print('Minimum Tweet ID per day:')
# print()
# print(df_SanSiro.groupby('date').min('id')['id'])
# print()
# print()
# print('Maximum Tweet ID per day:')
# print()
# print(df_SanSiro.groupby('date').max('id')['id'])

AttributeError: Can only use .dt accessor with datetimelike values

In [11]:
# Create a DataFrame with the authors of the tweets and their respective frequency
freq_authors = tweets['author'].value_counts()
freq_authors.head()

PianetaMilan     10
cmdotcom          5
sportface2016     4
MiTomorrow        4
Pradivio          4
Name: author, dtype: int64

In [12]:
# Extract all hashtags from the full text
tweets['hashtags_list'] = tweets['text'].apply(lambda x: re.findall(r"#(\w+)", x))

# Extract all mentions from the full text
tweets['mentions'] = tweets['text'].apply(lambda x: re.findall(r"@(\w+)", x))

# \w matches any single letter, number or underscore (same as [a zA Z0 9_])

#### Data Cleaning

NLTK - Natural Language ToolKit is a platform for building Python programs to work with human language data. It provides easy to use interfaces to over 50 corpora and lexical resources, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning.

In [13]:
import nltk
from nltk import FreqDist
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Text Pre-Processing

In [14]:
def preprocess_data(data):
    # remove numbers and turning words into lower case
    data = data.astype(str).str.replace('\d+','')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    
    # token lemmatization (ex. goes --> go)
    def lemmatize_text(text):
        return[(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    # remove punctuation
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

In [15]:
# apply preprocess_data function
pre_tweets = preprocess_data(tweets['text'])
tweets['text_proc'] = pre_tweets

# delete italian stopwords
stop_words = set(stopwords.words('italian'))
tweets['text_proc'] = tweets['text_proc'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\FC\AppData\Local\Temp\ipykernel_14404\3179111472.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+','')


## Analysis

#### Social Content Analysis

_Sentiment Analysis_

When dealing with social media text, we usually want to identify urls, hashtags, smileys as separate objects and do not tokenize it to individual characters.

_Emotion Analysis_

#### Social Network Analysis

Measures of Centrality

Community Detection

## Visualization